In [1]:
import pandas, os, numpy as np
import os, sklearn, pandas, numpy as np
from sklearn import svm
import skimage
from skimage import io, filters
from utils import utils # custom functions, in local environment
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## NN libs
import keras
from sklearn.decomposition import PCA
from keras.utils import to_categorical
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Conv3D, MaxPool2D, Dropout, Flatten

Using TensorFlow backend.


In [3]:
import data, config, tfidf
from utils import io

 Dataset :: namedtuple(
  'info': pandas.df
  'labels': pandas.df('filename.txt': 'genre')
  'genres': ['genre'] # unique genres
  'label_dataset': SubDataset
  'sentiment_dataset': SubDataset
  'book_sentiment_words_list': ['filename']

 SubDataset :: namedtuple(
   'dict_index_to_label' = dict to convert label_index -> label_name
   'dict_label_to_index'= dict to convert label_name -> label_index



In [4]:
# info = pandas.read_csv(config.dataset_dir + 'final_data.csv')
dataset = data.init_dataset()
dataset.info.keys()

Index(['title', 'filename', 'title.1', 'author', 'release year', 'genre',
       'pos score', 'neg score', 'neu score', 'comp score', 'amt pos',
       'amt neg'],
      dtype='object')

In [5]:
dataset.info

,title,filename,title.1,author,release year,genre,pos score,neg score,neu score,comp score,amt pos,amt neg
0,Jonah,3678.txt,Jonah,Louis Stone,"January, 2003",Poetry,0.063752,0.064028,0.670289,0.001590,2058,2017
1,The Wind in the Willows,289.txt,The Wind in the Willows,Kenneth Grahame,"July, 1995",Children's literature,0.075622,0.049711,0.708466,0.050425,1684,1172
2,Akbar,40155-8.txt,Akbar,P. A. S. van Limburg-Brouwer,"July 7, 2012",Syair,0.088927,0.047641,0.701412,0.078233,2988,1652
3,Erewhon,1906.txt,Erewhon,Samuel Butler,"March 20, 2005",Satire,0.075006,0.061587,0.750951,0.025886,2506,1952
4,Night and Day,1245.txt,Night and Day,Virginia Woolf,"March, 1998",Romance novel,0.070135,0.048907,0.711205,0.043948,4822,3088
5,She and Allan,5745.txt,She and Allan,H. Rider Haggard,"April 22, 2006",Fantasy novel,0.067608,0.057010,0.693447,0.020357,3234,2652
6,The Amateur Cracksman,706.txt,The Amateur Cracksman,E. W. Hornung,"November, 1996",Crime fiction,0.059576,0.045972,0.663071,0.026517,1363,1072
7,Anne of Geierstein,44247-8.txt,Anne of Geierstein,Walter Scott,"November 21, 2013",Historical novel,0.080691,0.054442,0.716062,0.048409,3268,2205
8,A Tramp Abroad,5786.txt,A Tramp Abroad,Mark Twain (Samuel Clemens),March 1994,Travel literature,0.062378,0.037464,0.720202,0.047015,642,387
9,Captives of the Flame,41905.txt,Captives of the Flame,Samuel R. Delany,"January 24, 2013",Science fantasy,0.037854,0.037340,0.663475,0.001440,850,776


In [6]:
name = '7813.txt'
data.get_label(name, dataset.labels)

'Novella'

In [7]:
dataset.labels.keys()
dataset.labels['1005.txt'][0]

'Christian fiction'

In [8]:
train = dataset.book_sentiment_words_list[:10]
test = dataset.book_sentiment_words_list[10:20]

In [9]:
x_train, y_train = data.extract_all(dataset, train)
x_test, y_test = data.extract_all(dataset, test)

In [10]:
y_train

['Christian fiction',
 'Adventure novel',
 'Western novel',
 'Travel literature',
 'Comic novel',
 'Künstlerroman',
 'Adventure novel',
 'Fantasy novel',
 "Children's literature",
 'Adventure novel']

### Prepare the labels

Encode the labels to one-hot vectors

In [11]:
y_train, y_test = data.labels_to_vectors(dataset.label_dataset,y_train, y_test)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Convert tokenlist to vectors

### Using sentiment-wordlist

In [12]:
b = dataset.sentiment_dataset
a = 'a s d fp sks dk fsji sdf'.split(' ')
data.tokenlist_to_vector(a,b)

array([0.57735027, 0.        , 0.        ])

### Using polarization scores

In [13]:
data.polarization_scores_to_vector(dataset,'706.txt')

array([0.05957593, 0.04597156, 0.66307096, 0.02651661])

### Using NER scores

In [14]:
# ...

## Train a model
Although most of the input consists of classes, many classes will be correlated (e.g. related genres).
Therefore a neural network is chosen to model the data

In [25]:
def to_vector(x_train, dataset):
    x2 = []
    for filename, tokens in x_train:
        v1 = data.tokenlist_to_vector(tokens, dataset.sentiment_dataset)
        v2 = data.polarization_scores_to_vector(dataset, filename)
        v = np.append(v1, v2)
        x2.append(v)
    return np.stack(x2)

In [27]:
x_train2 = to_vector(x_train, dataset)
x_train2.shape

(10, 7)

In [29]:
n_samples = len(x_train2) # = length of the list of images (matrices)
input_shape = x_train2.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label
print(n_samples, input_shape)
print('output length', output_length)

10 (7,)
output length 158


In [54]:
def sequential(input_shape, output_length):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape))
    model.add(Activation('relu'))  # because relu is awesome
    model.add(Dense(output_length))
    model.add(Activation('softmax'))
    # in addition, return a function that displays information about the model
    return model, model.summary

In [55]:
# import models # src/models.py

model, summary = sequential(input_shape, output_length)
summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               1024      
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 158)               20382     
_________________________________________________________________
activation_4 (Activation)    (None, 158)               0         
Total params: 21,406
Trainable params: 21,406
Non-trainable params: 0
_________________________________________________________________


## Loss function
- Categorical cross-entropy loss

In [56]:
learning_rate = 0.01
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Finally train the model

In [57]:
# Compute a batch size
batch_size = round(n_samples / 10)
# n epochs = n iterations over all the training data
epochs = 5

In [58]:
model.fit(x_train2, y_train, epochs=epochs, batch_size=batch_size, validation_split=1/5)

Train on 8 samples, validate on 2 samples
Epoch 1/5
8/8 [==============================] - 0s 57ms/step - loss: 5.0421 - acc: 0.0000e+00 - val_loss: 4.8717 - val_acc: 0.0000e+00
Epoch 2/5
8/8 [==============================] - 0s 2ms/step - loss: 4.1552 - acc: 0.1250 - val_loss: 4.3556 - val_acc: 0.0000e+00
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 2.8459 - acc: 0.1250 - val_loss: 4.4335 - val_acc: 0.0000e+00
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 2.1163 - acc: 0.1250 - val_loss: 5.5491 - val_acc: 0.0000e+00
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 2.0267 - acc: 0.2500 - val_loss: 6.1588 - val_acc: 0.5000


In [59]:
result_known_data = model.predict(x_train2)
result_known_data == y_train

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [60]:
result_known_data

array([[1.8631421e-05, 2.3759421e-05, 1.9479583e-05, ..., 1.8151064e-05,
        2.3767081e-05, 6.9395676e-02],
       [2.3281935e-05, 2.9353312e-05, 2.4164980e-05, ..., 2.2704815e-05,
        2.9492827e-05, 7.0404150e-02],
       [3.2635868e-05, 4.0770607e-05, 3.4019729e-05, ..., 3.1939140e-05,
        4.0833176e-05, 7.2843209e-02],
       ...,
       [2.9167664e-05, 3.6507561e-05, 3.0199690e-05, ..., 2.8357514e-05,
        3.6741229e-05, 7.1583681e-02],
       [1.3587793e-05, 1.7434430e-05, 1.4107607e-05, ..., 1.3219166e-05,
        1.7522842e-05, 6.7066565e-02],
       [3.0514009e-05, 3.8215148e-05, 3.1803043e-05, ..., 2.9817957e-05,
        3.8283291e-05, 7.2381467e-02]], dtype=float32)